# Algorithm 16: Template Pair Stack

The Template Pair Stack processes template structure information to produce pair features that are combined with the main pair representation.

## Algorithm Pseudocode

![TemplatePairStack](../imgs/algorithms/TemplatePairStack.png)

## Source Code Location
- **File**: `AF2-source-code/model/modules.py`
- **Class**: `TemplatePairStack`, `TemplatePairStackIteration`
- **Lines**: 522-623

## Purpose

Templates are experimentally determined structures of related proteins. The Template Pair Stack:

1. Processes template pairwise features
2. Uses triangle multiplication and attention (similar to Evoformer)
3. Produces template-derived pair information

In [ ]:
import numpy as np

np.random.seed(42)

In [ ]:
def template_pair_stack_iteration(t, c_t=64, num_blocks=2):
    """
    One iteration of Template Pair Stack - Algorithm 16.
    
    Similar to Evoformer but simpler, for template features.
    
    Args:
        t: Template pair features [N_templ, N_res, N_res, c_t]
        c_t: Template channel dimension
        num_blocks: Number of iterations
    
    Returns:
        Updated template features
    """
    N_templ, N_res, _, c_t = t.shape
    
    print(f"Template Pair Stack")
    print(f"  Templates: {N_templ}")
    print(f"  Residues: {N_res}")
    print(f"  Channels: {c_t}")
    
    for block_idx in range(num_blocks):
        print(f"  Block {block_idx + 1}:")
        
        for templ_idx in range(N_templ):
            z = t[templ_idx]  # [N_res, N_res, c_t]
            
            # Step 1: Triangle Multiplication (Outgoing)
            # Simplified: just a residual update
            z_norm = (z - z.mean(axis=-1, keepdims=True)) / (z.std(axis=-1, keepdims=True) + 1e-5)
            z = z + z_norm * 0.1
            
            # Step 2: Triangle Multiplication (Incoming)
            z_norm = (z - z.mean(axis=-1, keepdims=True)) / (z.std(axis=-1, keepdims=True) + 1e-5)
            z = z + z_norm * 0.1
            
            # Step 3: Triangle Attention (Starting)
            z = z + np.random.randn(*z.shape) * 0.01
            
            # Step 4: Triangle Attention (Ending)
            z = z + np.random.randn(*z.shape) * 0.01
            
            # Step 5: Pair Transition
            z_norm = (z - z.mean(axis=-1, keepdims=True)) / (z.std(axis=-1, keepdims=True) + 1e-5)
            z = z + z_norm * 0.1
            
            t[templ_idx] = z
        
        print(f"    Processed {N_templ} templates")
    
    return t

In [ ]:
# Test
N_templ, N_res, c_t = 4, 32, 64
t = np.random.randn(N_templ, N_res, N_res, c_t)

print("Test Template Pair Stack")
print("="*50)
print(f"Input shape: {t.shape}")

output = template_pair_stack_iteration(t.copy(), num_blocks=2)
print(f"\nOutput shape: {output.shape}")
print(f"Shape preserved: {output.shape == t.shape}")

## Source Code Reference

```python
# From AF2-source-code/model/modules.py

class TemplatePairStack(hk.Module):
  """Pair stack for the templates.

  Jumper et al. (2021) Suppl. Alg. 16 "TemplatePairStack"
  """

  def __call__(self, pair_act, pair_mask, is_training, ...):
    # Apply template pair stack iteration
    for _ in range(self.num_block):
      pair_act = TemplatePairStackIteration(...)(pair_act, pair_mask, ...)
    return pair_act
```